# Exercise 2: Creating Redshift Cluster using the AWS python SDK 
## An example of Infrastructure-as-code

In [1]:
import pandas as pd
import boto3
import json

# STEP 0: Make sure you have an AWS secret and access key

- Create a new IAM user in your AWS account
- Give it `AdministratorAccess`, From `Attach existing policies directly` Tab
- Take note of the access key and secret 
- Edit the file `dwh.cfg` in the same folder as this notebook and fill
<font color='red'>
<BR>
[AWS]<BR>
KEY= YOUR_AWS_KEY<BR>
SECRET= YOUR_AWS_SECRET<BR>
<font/>


# Load DWH Params from a file

In [2]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dwhCluster
4,DWH_DB,dwh
5,DWH_DB_USER,dwhuser
6,DWH_DB_PASSWORD,Passw0rd
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,dwhRole


## Create clients for EC2, S3, IAM, and Redshift

In [3]:
import boto3

ec2 = boto3.resource('ec2',region_name = 'us-west-2',aws_access_key_id = KEY, aws_secret_access_key = SECRET )  

#low level boto3
#s3 = boto3.client('s3',region_name = 'us-west-2',aws_access_key_id = KEY, aws_secret_access_key = SECRET )

#high level boto3
s3 = boto3.resource('s3',region_name = 'us-west-2',aws_access_key_id = KEY, aws_secret_access_key = SECRET )

iam = boto3.client('iam',region_name = 'us-west-2',aws_access_key_id = KEY, aws_secret_access_key = SECRET )

redshift = boto3.client('redshift',region_name = 'us-west-2',aws_access_key_id = KEY, aws_secret_access_key = SECRET )

## Check out the sample data sources on S3

In [18]:
sampleDbBucket =  s3.Bucket("awssampledbuswest2")
# TODO: Iterate over bucket objects starting with "ssbgz" and print

for object_summary in sampleDbBucket.objects.filter(Prefix="ssbgz/"):
    #print(object_summary.key)
    print(object_summary)

s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/customer0002_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/dwdate.tbl.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0000_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0001_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0002_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0003_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0004_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0005_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0006_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0007_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='s

## STEP 1: IAM ROLE
- Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

In [5]:
def create_role(role_name, allowed_services):
    """
    Creates a role that lets a list of specified services assume the role.

    :param role_name: The name of the role.
    :param allowed_services: The services that can assume the role.
    :return: The newly created role.
    """
    trust_policy = {
        'Version': '2012-10-17',
        'Statement': [{
                'Effect': 'Allow',
                'Principal': {'Service': service},
                'Action': 'sts:AssumeRole'
            } for service in allowed_services
        ]
    }

    try:
        role = iam.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy))
        logger.info("Created role %s.", role.name)
    except ClientError:
        logger.exception("Couldn't create role %s.", role_name)
        raise
    else:
        return role

In [6]:
trust_policy = {
        'Version': '2012-10-17',
        'Statement': [{
                'Effect': 'Allow',
                'Principal': {'Service': 'redshift.amazonaws.com'},
                'Action': 'sts:AssumeRole'
        }]
}

In [7]:
# TODO: Create the IAM role

try:
    print('1.1 Creating a new IAM Role')
    dwhRole = iam.create_role(Path = '/', RoleName = DWH_IAM_ROLE_NAME,
                             Description = "Thist is my firs test",
                             AssumeRolePolicyDocument = json.dumps(trust_policy)
                             ) 
    print(dwhRole)
    

except Exception as e:
    print(e)

1.1 Creating a new IAM Role
{'Role': {'Path': '/', 'RoleName': 'dwhRole', 'RoleId': 'AROAZZK5UHLWRPR36F5HK', 'Arn': 'arn:aws:iam::672895613677:role/dwhRole', 'CreateDate': datetime.datetime(2022, 8, 6, 8, 6, 14, tzinfo=tzutc()), 'AssumeRolePolicyDocument': {'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': {'Service': 'redshift.amazonaws.com'}, 'Action': 'sts:AssumeRole'}]}}, 'ResponseMetadata': {'RequestId': 'd4d2d1e5-f0a3-416c-b3c6-3824db161f1a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd4d2d1e5-f0a3-416c-b3c6-3824db161f1a', 'content-type': 'text/xml', 'content-length': '770', 'date': 'Sat, 06 Aug 2022 08:06:14 GMT'}, 'RetryAttempts': 0}}


In [8]:
# TODO: Attach Policy
print('1.2 Attaching Policy')
iam.attach_role_policy(RoleName = DWH_IAM_ROLE_NAME,
                      PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
                      )['ResponseMetadata']['HTTPStatusCode']



1.2 Attaching Policy


200

In [9]:
# TODO: Get and print the IAM role ARN
print('1.3 Get the IAM role ARN')
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

#roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)
print(roleArn)

1.3 Get the IAM role ARN
arn:aws:iam::672895613677:role/dwhRole


## STEP 2:  Redshift Cluster

- Create a RedShift Cluster
- For complete arguments to `create_cluster`, see [docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster)

In [10]:
try:
    response = redshift.create_cluster(        
        # TODO: add parameters for hardware
        ClusterType = DWH_CLUSTER_TYPE,
        NodeType = DWH_NODE_TYPE,
        NumberOfNodes = int(DWH_NUM_NODES),

        # TODO: add parameters for identifiers & credentials
        DBName = DWH_DB,
        ClusterIdentifier = DWH_CLUSTER_IDENTIFIER,
        MasterUsername = DWH_DB_USER,
        MasterUserPassword = DWH_DB_PASSWORD,
        
        
        # TODO: add parameter for role (to allow s3 access)
         IamRoles = [roleArn]
    )
except Exception as e:
    print(e)

In [11]:
response

{'Cluster': {'ClusterIdentifier': 'dwhcluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'creating',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'dwhuser',
  'DBName': 'dwh',
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-007d708156c532464',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-04fc4170bbd3cfd02',
  'PreferredMaintenanceWindow': 'mon:11:30-mon:12:00',
  'PendingModifiedValues': {'MasterUserPassword': '****'},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [],
  'EnhancedVpcRouting': False,
  'IamRoles': [{'IamRoleArn': 'arn:aws:iam::672895613677:role/dwhRole',
    'ApplyStatus': 'adding'}],
  'Mai

## 2.1 *Describe* the cluster to see its status
- run this block several times until the cluster status becomes `Available`

In [14]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

/var/folders/s8/1mp0j0gj3fz3qc0nkwt9k5fc0000gn/T/ipykernel_75429/1230814843.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.cotpgb4thtqp.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-04fc4170bbd3cfd02
7,NumberOfNodes,4


In [13]:
myClusterProps

{'ClusterIdentifier': 'dwhcluster',
 'NodeType': 'dc2.large',
 'ClusterStatus': 'creating',
 'ClusterAvailabilityStatus': 'Modifying',
 'MasterUsername': 'dwhuser',
 'DBName': 'dwh',
 'AutomatedSnapshotRetentionPeriod': 1,
 'ManualSnapshotRetentionPeriod': -1,
 'ClusterSecurityGroups': [],
 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-007d708156c532464',
   'Status': 'active'}],
 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
   'ParameterApplyStatus': 'in-sync'}],
 'ClusterSubnetGroupName': 'default',
 'VpcId': 'vpc-04fc4170bbd3cfd02',
 'PreferredMaintenanceWindow': 'mon:11:30-mon:12:00',
 'PendingModifiedValues': {'MasterUserPassword': '****'},
 'ClusterVersion': '1.0',
 'AllowVersionUpgrade': True,
 'NumberOfNodes': 4,
 'PubliclyAccessible': True,
 'Encrypted': False,
 'ClusterNodes': [],
 'Tags': [],
 'EnhancedVpcRouting': False,
 'IamRoles': [{'IamRoleArn': 'arn:aws:iam::672895613677:role/dwhRole',
   'ApplyStatus': 'adding'}],
 'MaintenanceTrackName

<h2> 2.2 Take note of the cluster <font color='red'> endpoint and role ARN </font> </h2>

<font color='red'>DO NOT RUN THIS unless the cluster status becomes "Available" </font>

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
print(DWH_ENDPOINT)
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)
print("DWH_ROLE_ARN :: ", roleArn)

## STEP 3: Open an incoming  TCP port to access the cluster ednpoint

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name ,  
        CidrIp='0.0.0.0/0',  
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

## STEP 4: Make sure you can connect to the clusterConnect to the cluster

In [ ]:
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string

## STEP 5: Clean up your resources

<b><font color='red'>DO NOT RUN THIS UNLESS YOU ARE SURE <br/> 
    We will be using these resources in the next exercises</span></b>

In [15]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

{'Cluster': {'ClusterIdentifier': 'dwhcluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'dwhuser',
  'DBName': 'dwh',
  'Endpoint': {'Address': 'dwhcluster.cotpgb4thtqp.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2022, 8, 6, 8, 8, 30, 91000, tzinfo=tzutc()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-007d708156c532464',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-04fc4170bbd3cfd02',
  'AvailabilityZone': 'us-west-2d',
  'PreferredMaintenanceWindow': 'mon:11:30-mon:12:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'Publ

- run this block several times until the cluster really deleted

In [16]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

/var/folders/s8/1mp0j0gj3fz3qc0nkwt9k5fc0000gn/T/ipykernel_75429/1230814843.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,deleting
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.cotpgb4thtqp.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-04fc4170bbd3cfd02
7,NumberOfNodes,4


In [17]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!

{'ResponseMetadata': {'RequestId': '740d44e5-cc13-4e6f-95b5-d98cbc05e3ae',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '740d44e5-cc13-4e6f-95b5-d98cbc05e3ae',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Sat, 06 Aug 2022 09:16:37 GMT'},
  'RetryAttempts': 0}}